该ipynb未完成，目前的问题：

https://chatgpt.com/s/t_6992ef18f3c08191a88814dd6f668b9a
https://chatgpt.com/s/t_6992ef369e948191bba754b0ce532ba6
https://chatgpt.com/s/t_6992ef4e91988191a3e98c8944d6434f

# GRPO：仅用 Embedding 相似度奖励（你的 Qwen3-4B-Thinking 训练后整模 + Qwen3-VL-Embedding-2B）

变化点（相对你之前看的 Unslo†h notebook）：
1) 数据集：读取 jsonl，每行一个 `messages`（OpenAI messages），单轮对话
2) prompt：直接用 `messages[:-1]`（不额外加 system）
3) reward：对 completion 做 “剥离思维链后的最终输出” -> embedding -> 与参考答案 embedding 相似度
4) 工程：同一张卡上跑两个 vLLM（生成 + embedding），必须拆分 `gpu_memory_utilization`


In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
# =======================
# 1) 安装依赖
# =======================
!pip -q install --upgrade pip
!pip -q install -U "unsloth" "unsloth-zoo" \
  "vllm==0.11.2" \
  "trl==0.22.2" \
  "transformers==4.57.1" \
  "qwen-vl-utils>=0.0.14" "datasets" "accelerate"


In [ ]:
!pip install modelscope

In [ ]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('unsloth/Qwen3-Embedding-0.6B', local_dir="./Qwen3-Embedding-0.6B")

In [ ]:
import os
os.environ["VLLM_USE_V1"] = "0"  # 禁用 vLLM V1，引导走旧引擎，新引擎与trl不兼容


In [ ]:
import os

# 只从本地读取模型文件，防止hf连不上
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"


In [ ]:
import os

# 防止pytorch保留太多显存
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# =======================
# 2) 基本配置（按你本地改路径）
# =======================
import os
import re
import gc
import math
import numpy as np
import torch
from datasets import load_dataset

# 你的整模（已 SFT 全量微调过，并保存成 vLLM/HF 能加载的目录）
GEN_MODEL_PATH = "/root/autodl-tmp/lora_model" # lora_model文件夹存的是完整权重，只是名字是lora_model

# 你的 jsonl：每行一个对象，至少包含 {"messages": [...]}（单轮对话）
DATA_JSONL_PATH = "/root/autodl-tmp/dataset.segmented.jsonl"

# embedding 模型
EMB_MODEL_NAME = "./Qwen3-Embedding-0.6B"

# ===== 显存关键：两个 vLLM 实例要拆分占用 =====
# 32GB 参考：生成 0.65~0.75，embedding 0.20~0.30 之间调
GEN_GPU_MEM_UTIL = 0.65
EMB_GPU_MEM_UTIL = 0.20

# ===== 训练相关 =====
max_seq_length = 8192
lora_rank = 128

num_generations = 4
max_steps = 12000
save_steps = 1200

per_device_train_batch_size = 1
gradient_accumulation_steps = 1

temperature = 1.0
top_p = 1.0

# KL 约束：建议很小，防止跑偏
beta_kl = 0.05

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device =", device)


In [ ]:
# =======================
# 3) 加载 embedding 模型（Qwen3-Embedding-0.6B / vLLM task=embed）
# =======================
import torch
import torch.nn.functional as F
from vllm import LLM

class VLLMTextEmbedder:
    """
    Qwen3-Embedding-* 推荐用纯文本输入：
      "Instruct: ...\nQuery: ..."
    vLLM 侧直接 task="embed"，不需要 runner="pooling" / apply_chat_template。
    """
    def __init__(
        self,
        model_name: str,
        dtype: str = "bfloat16",
        gpu_memory_utilization: float = 0.25,
        instruction: str = "Represent the text for semantic similarity.",
        max_model_len: int = 8192,   #  Qwen3-Embedding-0.6B 支持 32k，但不需要那么多，所以设小点，省显存
    ):
        self.instruction = instruction

        self.llm = LLM(
            model=model_name,
            task="embed",
            dtype=dtype,
            trust_remote_code=True,
            gpu_memory_utilization=gpu_memory_utilization,
            max_model_len=max_model_len,
        )

    def _format(self, text: str) -> str:
        # ⚠️ 为了让“参考答案 embedding”和“模型输出 embedding”在同一空间，
        # 建议两边都用同一个格式（要么都带 instruction，要么都不带）。
        if self.instruction and self.instruction.strip():
            return f"Instruct: {self.instruction}\nQuery: {text}"
        return text

    @torch.no_grad()
    def embed_texts(self, texts, normalize=True):
        inputs = [self._format(t) for t in texts]
        outs = self.llm.embed(inputs)
        embs = torch.tensor([o.outputs.embedding for o in outs],
                            dtype=torch.float32,
                            device="cuda" if torch.cuda.is_available() else "cpu")
        if normalize:
            embs = F.normalize(embs, p=2, dim=-1)
        return embs


# 建议按官方推荐：instruction 用英文（训练时大多是英文 instruction）
EMB_INSTRUCTION = "Represent the text for semantic similarity."

embedder = VLLMTextEmbedder(
    EMB_MODEL_NAME,
    dtype="bfloat16",
    gpu_memory_utilization=EMB_GPU_MEM_UTIL,
    instruction=EMB_INSTRUCTION,
    max_model_len=max_seq_length, # 比grpo时模型的输出(max_seq_length - max_prompt_length)长一点，防止模型输出超过Embedder的上下文上限
)

print("Loaded embedding model.")


In [ ]:
# =======================
# 4) 加载生成模型（Unsloth + LoRA）
# =======================
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = GEN_MODEL_PATH,
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = True,                 # vLLM rollout
    max_lora_rank = lora_rank,
    gpu_memory_utilization = GEN_GPU_MEM_UTIL,  # 关键：给生成模型的 vLLM 限额
    # local_files_only = True,
)

# TRL 建议 left padding（见 TRL dataset/grpo 说明）
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
    ],
    lora_alpha = lora_rank * 2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("Loaded gen model + LoRA.")


In [ ]:
# =======================
# 5) 读取 jsonl messages 数据集，并转成 GRPOTrainer 格式
#    修复点：map 时 remove_columns，把原始 "messages"（以及其它原始列）删掉
# =======================
from datasets import load_dataset

raw0 = load_dataset("json", data_files=DATA_JSONL_PATH, split="train")
orig_cols = raw0.column_names  # 通常包含 ["messages"]，也可能还有别的列

def to_grpo_format(example, idx):
    msgs = example.get("messages", None)

    # 防御：坏样本直接变空，后面 filter 会丢掉
    if not isinstance(msgs, list) or len(msgs) < 2:
        return {"row_id": idx, "prompt": [], "answer_text": ""}

    # prompt：去掉最后一条 assistant（参考答案）
    prompt_msgs = msgs[:-1]

    # 可选：如果你坚决不要 system，就过滤掉
    prompt_msgs = [m for m in prompt_msgs if m.get("role") != "system"]

    # 参考答案：最后一条 assistant
    answer_msg = msgs[-1]
    if isinstance(answer_msg, dict):
        answer_text = str(answer_msg.get("content", ""))
    else:
        answer_text = str(answer_msg)

    return {
        "row_id": idx,
        "prompt": prompt_msgs,      # TRL 支持：prompt 直接用 messages 列表
        "answer_text": answer_text,
    }

raw = raw0.map(
    to_grpo_format,
    with_indices=True,
    remove_columns=orig_cols,      # 关键：删掉原始列，避免同时存在 messages + prompt
    desc="Convert to GRPO format",
)

# 过滤空样本
raw = raw.filter(lambda x: len(x["prompt"]) > 0 and len(x["answer_text"].strip()) > 0)

print("keys:", raw.column_names)  # 期望：['row_id','prompt','answer_text']
print(raw[0])
print("N =", len(raw))


In [ ]:
# =======================
# 7) 剥离思维链 + reward（embedding 相似度）
# =======================

THINK_END_TAGS = [
    "</think>",
    "<end_working_out>",
]

def strip_thought(text: str, tail_chars: int = 3000) -> str:
    t = text

    # 命中 </think> / <end_working_out> 时：
    # - 若 tag 之后确实有内容：返回“全部内容”（不再截断 3000）
    # - 若 split 后为空（或全是空白）：保持原逻辑（走尾部 tail_chars 回退）
    for tag in THINK_END_TAGS:
        if tag in t:
            after = t.split(tag, 1)[1]
            after_stripped = after.strip()
            if after_stripped:
                return after_stripped
            t = after  # 为空则继续走原逻辑回退
            break

    # 无 tag：保持原始逻辑不变（只取尾部 tail_chars）
    t = t.strip()
    t = t[-tail_chars:].strip() if t else text[-tail_chars:].strip()
    return t

# -----------------------
# tokenizer 版 repetition penalty
# -----------------------
def repetition_penalty_from_ids(token_ids, min_tokens: int = 20) -> float:
    """
    用 token_id bigram 多样性检测复读：
    ratio = uniq_bigrams / total_bigrams
    """
    if not token_ids:
        return -2.0

    n = len(token_ids)
    if n < min_tokens:
        return 0.0

    total = n - 1
    if total <= 0:
        return 0.0

    uniq = len({(token_ids[i], token_ids[i + 1]) for i in range(total)})
    ratio = uniq / max(1, total)

    if ratio < 0.3:
        return -2.0
    if ratio < 0.5:
        return -1.0
    return 0.0

# -----------------------
# completions_ids 版 repetition penalty（无需 tokenizer.encode）
# - 尽量模拟你当前 strip_thought 的“tag 后内容”逻辑
# - 若找不到 tag，则退化为取尾部 tail_tokens
# -----------------------
def _find_subseq_end(seq, pattern):
    """返回 pattern 在 seq 中首次出现的位置的“结束下标”(i+len(pattern))；找不到返回 -1。"""
    m = len(pattern)
    if m == 0:
        return -1
    # 朴素搜索，completion 一般不长，足够用
    for i in range(0, len(seq) - m + 1):
        if seq[i : i + m] == pattern:
            return i + m
    return -1

def repetition_penalty_completions_ids_batch(
    completions_ids,
    tokenizer,
    think_end_tag_id_seqs,
    empty_mask,
    min_tokens: int = 20,
    tail_tokens: int = 512,
):
    """
    输入：
      completions_ids: list[list[int]] 或 torch.Tensor(B, L)
    输出：
      penalties: list[float]
    """
    # 兼容 tensor
    if isinstance(completions_ids, torch.Tensor):
        ids_list = completions_ids.detach().cpu().tolist()
    else:
        ids_list = completions_ids

    pad_id = getattr(tokenizer, "pad_token_id", None)
    eos_id = getattr(tokenizer, "eos_token_id", None)

    penalties = []
    for ids, is_empty in zip(ids_list, empty_mask):
        if is_empty:
            penalties.append(-2.0)
            continue

        # 清掉 padding / -100（有些实现会用 -100 做 ignore）
        cleaned = []
        for t in ids:
            if t == -100:
                continue
            if pad_id is not None and t == pad_id:
                continue
            cleaned.append(t)

        # 去掉末尾 eos（避免 eos 重复影响 bigram）
        if eos_id is not None:
            while cleaned and cleaned[-1] == eos_id:
                cleaned.pop()

        # 找 </think> / <end_working_out> 等 tag 的 token 序列，截取 tag 后内容
        cut_pos = -1
        for tag_ids in think_end_tag_id_seqs:
            p = _find_subseq_end(cleaned, tag_ids)
            if p != -1:
                cut_pos = p
                break  # 命中一个就用第一个命中的

        if cut_pos != -1:
            eff = cleaned[cut_pos:]
        else:
            eff = cleaned[-tail_tokens:] if tail_tokens > 0 else cleaned

        penalties.append(repetition_penalty_from_ids(eff, min_tokens=min_tokens))

    return penalties

def repetition_penalty_tokenizer_batch(
    texts,
    tokenizer,
    min_tokens: int = 20,
    tail_tokens: int = 512,   # 只看尾部 N tokens：更贴近“复读常发生在结尾”，也更省时
):
    """
    批量用 tokenizer 编码，然后对每条算 repetition penalty。
    - add_special_tokens=False：避免特殊 token 干扰统计
    - truncation_side='left'：保留尾部 token（更有价值）
    """
    # 空文本沿用你旧逻辑：直接 -2.0
    empty_mask = [(t is None) or (len(str(t).strip()) == 0) for t in texts]
    safe_texts = [("" if e else str(t)) for t, e in zip(texts, empty_mask)]

    # 临时把 truncation_side 设为 left，确保 max_length 截断时保留尾部 token
    old_trunc_side = getattr(tokenizer, "truncation_side", "right")
    tokenizer.truncation_side = "left"
    enc = tokenizer(
        safe_texts,
        add_special_tokens=False,
        truncation=True,
        max_length=tail_tokens,
        padding=False,
        return_attention_mask=False,
        return_token_type_ids=False,
    )
    tokenizer.truncation_side = old_trunc_side

    ids_list = enc["input_ids"]

    penalties = []
    for ids, is_empty in zip(ids_list, empty_mask):
        if is_empty:
            penalties.append(-2.0)
        else:
            penalties.append(repetition_penalty_from_ids(ids, min_tokens=min_tokens))
    return penalties

@torch.no_grad()
def reward_embedding_similarity(prompts, completions, row_id, answer_text, completions_ids=None, **kwargs):
    # completions: list[ list[{role, content}] ]（通常是 assistant message）
    comp_texts = []
    for c in completions:
        if isinstance(c, str):
            comp_texts.append(c)
        else:
            comp_texts.append(c[0]["content"])

    final_texts = [strip_thought(t) for t in comp_texts]
    empty_mask = [1 if len(t.strip()) == 0 else 0 for t in final_texts]

    # 1) 模型输出 embedding（每条 completion 都要算）
    emb_out = embedder.embed_texts(final_texts, normalize=True)

    # 2) 参考答案 embedding：按 batch 去重计算，然后复用到 num_generations
    #    row_id 会在 GRPO 中重复（每条 prompt 对应 num_generations 个 completion）
    rid2pos = {}
    uniq_answers = []
    for rid, ans in zip(row_id, answer_text):
        if rid not in rid2pos:
            rid2pos[rid] = len(uniq_answers)
            uniq_answers.append(ans)

    emb_tgt_uniq = embedder.embed_texts(uniq_answers, normalize=True)

    idx = torch.tensor(
        [rid2pos[rid] for rid in row_id],
        device=emb_tgt_uniq.device,
        dtype=torch.long,
    )
    emb_tgt = emb_tgt_uniq.index_select(0, idx)

    # cosine（两边都 normalize 过）
    sim = (emb_out * emb_tgt).sum(dim=-1)
    reward = 10.0 * sim

    # 3) repetition penalty：优先用 TRL 传入的 completions_ids（无需 tokenizer）
    if completions_ids is None:
        completions_ids = kwargs.get("completions_ids", None)

    if completions_ids is not None:
        # 把 think end tag 编成 token 序列
        think_end_tag_id_seqs = [tokenizer.encode(tag, add_special_tokens=False) for tag in THINK_END_TAGS]

        pen = repetition_penalty_completions_ids_batch(
            completions_ids=completions_ids,
            tokenizer=tokenizer,
            think_end_tag_id_seqs=think_end_tag_id_seqs,
            empty_mask=empty_mask,
            min_tokens=20,
            tail_tokens=512,
        )
    else:
        # fallback：如果没传 completions_ids，就退回 tokenizer 版
        pen = repetition_penalty_tokenizer_batch(
            final_texts,
            tokenizer,
            min_tokens=20,
            tail_tokens=512,
        )

    reward = reward + torch.tensor(pen, device=reward.device, dtype=reward.dtype)

    # 空输出额外惩罚（保留你原逻辑）
    reward = reward + torch.tensor(
        [-3.0 if e else 0.0 for e in empty_mask],
        device=reward.device,
        dtype=reward.dtype,
    )

    return reward.detach().cpu().tolist()


In [ ]:
# =======================
# 8) 自检 reward 是否合理
# =======================
sample = raw[0]
print("PROMPT:", sample["prompt"])
print("\nANSWER:\n", sample["answer_text"])

fake_completions = [
    [{"role":"assistant","content": f"</think>\n{sample['answer_text']}"}],
    [{"role":"assistant","content": "</think>\n我不太确定，但大致是另一个意思。"}],
]

r = reward_embedding_similarity(
    prompts=[sample["prompt"], sample["prompt"]],
    completions=fake_completions,
    row_id=[sample["row_id"], sample["row_id"]],
    answer_text=[sample["answer_text"], sample["answer_text"]],
)
print("\nRewards:", r)


In [ ]:
# =======================
# 在grpo正在训练的情况下，测试训练的效果
# =======================

import os
import json
import time
import uuid
import torch
from transformers import TrainerCallback

class OnDemandGenerateCallback(TrainerCallback):
    """
    单卡训练中按需推理（不会常驻占显存，不会频繁生成）：
    - 你往 request_path 追加一行 JSON（jsonl）= 投递一个生成请求
    - 训练在 step_end 的“安全时机”轮询到请求后，用当前训练权重 generate
    - 把结果追加写入 result_path（jsonl），并可选打印到 stdout
    - 支持 start_from_end：重启训练后默认从 requests 文件末尾开始消费，避免回放旧请求
    """
    def __init__(
        self,
        tokenizer,
        request_path="grpo_eval_requests.jsonl",
        result_path="grpo_eval_results.jsonl",
        poll_interval_sec=5.0,     # 多久检查一次 inbox（越小越“随时”，但开销略增）
        max_requests_per_poll=1,   # 单卡建议 1，避免一次生成太久拖训练
        default_gen_kwargs=None,
        enable_thinking=False,     # Qwen3-4B-Thinking-2507的模型卡说过它会始终输出思维链，所以不需要设置True
        start_from_end=True,       # 重启从文件末尾开始读
        print_outputs=True,        # 是否在训练日志里打印输出
        keep_full_text=True,      # True: 保存 full_text（含prompt）; False: 仅保存生成部分
    ):
        self.tokenizer = tokenizer
        self.request_path = request_path
        self.result_path = result_path
        self.poll_interval_sec = poll_interval_sec
        self.max_requests_per_poll = max_requests_per_poll
        self.default_gen_kwargs = default_gen_kwargs or dict(
            max_new_tokens=192,
            do_sample=True,
            temperature=0.7,
            top_p=0.8,
            top_k=20,
        )
        self.enable_thinking = enable_thinking
        self.print_outputs = print_outputs
        self.keep_full_text = keep_full_text

        self._last_poll_t = 0.0
        self._offset = 0

        # 重启后不回放历史请求：从文件末尾开始读
        if start_from_end and os.path.exists(self.request_path):
            self._offset = os.path.getsize(self.request_path)

    def _read_new_requests(self):
        if not os.path.exists(self.request_path):
            return []

        reqs = []
        with open(self.request_path, "r", encoding="utf-8") as f:
            f.seek(self._offset)
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    reqs.append(json.loads(line))
                except Exception:
                    reqs.append({"_bad_line": line[:300]})
                if len(reqs) >= self.max_requests_per_poll:
                    break
            self._offset = f.tell()
        return reqs

    def _append_result(self, obj):
        os.makedirs(os.path.dirname(self.result_path) or ".", exist_ok=True)
        with open(self.result_path, "a", encoding="utf-8") as f:
            f.write(json.dumps(obj, ensure_ascii=False) + "\n")

    def on_step_end(self, args, state, control, **kwargs):
        now = time.time()
        if now - self._last_poll_t < self.poll_interval_sec:
            return control
        self._last_poll_t = now

        reqs = self._read_new_requests()
        if not reqs:
            return control

        model = kwargs.get("model", None)
        if model is None:
            return control

        # 拿 device（比 model.device 更通用）
        try:
            device = next(model.parameters()).device
        except StopIteration:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        was_training = model.training
        model.eval()

        for req in reqs:
            if "_bad_line" in req:
                self._append_result({
                    "ok": False,
                    "step": int(state.global_step),
                    "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                    "error": f"bad_json_line: {req['_bad_line']}",
                })
                continue

            req_id = req.get("id") or f"manual_{uuid.uuid4().hex[:8]}"
            gen_kwargs = dict(self.default_gen_kwargs)
            gen_kwargs.update(req.get("gen", {}))

            # 允许两种请求格式：
            # 1) {"prompt": "..."}
            # 2) {"messages": [...]}
            if "messages" in req:
                messages = req["messages"]
            else:
                messages = [{"role": "user", "content": req.get("prompt", "")}]

            try:
                inputs = None
                out_ids = None
                gen_ids = None
                
                text = self.tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True,
                    enable_thinking=self.enable_thinking,
                )

                inputs = self.tokenizer(
                    text,
                    return_tensors="pt",
                ).to(device)

                prompt_len = inputs["input_ids"].shape[1]

                with torch.inference_mode():
                    out_ids = model.generate(**inputs, **gen_kwargs)

                # 只取生成部分（等价于 TextStreamer(skip_prompt=True) 的效果）
                gen_ids = out_ids[0, prompt_len:]
                gen_text = self.tokenizer.decode(gen_ids, skip_special_tokens=True)

                # 可选：也保存“剥离思维链后的版本”
                gen_text_stripped = strip_thought(gen_text)

                result = {
                    "ok": True,
                    "id": req_id,
                    "step": int(state.global_step),
                    "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                    "request": req,
                    "gen_text": gen_text,
                    "gen_text_stripped": gen_text_stripped,
                }

                if self.keep_full_text:
                    full_text = self.tokenizer.decode(out_ids[0], skip_special_tokens=True)
                    result["full_text"] = full_text

                self._append_result(result)

                if self.print_outputs:
                    print(f"\n\n===== [on-demand gen | step {state.global_step} | id={req_id}] =====")
                    print(gen_text_stripped)

            except torch.cuda.OutOfMemoryError:
                self._append_result({
                    "ok": False,
                    "id": req_id,
                    "step": int(state.global_step),
                    "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                    "error": "CUDA OOM: try smaller max_new_tokens / temperature / shorter prompt",
                })

            except Exception as e:
                self._append_result({
                    "ok": False,
                    "id": req_id,
                    "step": int(state.global_step),
                    "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                    "error": repr(e),
                })

            finally:
                del out_ids, inputs, gen_ids
                gc.collect()                

        # 缓解碎片
        torch.cuda.empty_cache()
        
        if was_training:
            model.train()

        return control

### 触发“在grpo正在训练的情况下，测试训练的效果”

```python
import json, time, uuid

REQUEST_PATH = "grpo_eval_requests.jsonl"

def enqueue_prompt(prompt: str, gen=None, req_id=None, request_path=REQUEST_PATH):
    req = {
        "id": req_id or f"manual_{int(time.time())}_{uuid.uuid4().hex[:6]}",
        "prompt": prompt,
        "gen": gen or {},
    }
    with open(request_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(req, ensure_ascii=False) + "\n")
    print("Queued:", req["id"])
    return req["id"]

def enqueue_messages(messages, gen=None, req_id=None, request_path=REQUEST_PATH):
    req = {
        "id": req_id or f"manual_{int(time.time())}_{uuid.uuid4().hex[:6]}",
        "messages": messages,
        "gen": gen or {},
    }
    with open(request_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(req, ensure_ascii=False) + "\n")
    print("Queued:", req["id"])
    return req["id"]

# 示例：随手投递一个请求
_ = enqueue_prompt(
    "用三句话解释什么是 overfitting，并举一个简单例子。",
    gen={"max_new_tokens": 160, "temperature": 0.6, "top_p": 0.9},
)
```

### 查询结果
```python
import json

RESULT_PATH = "grpo_eval_results.jsonl"

def read_last_results(n=3, result_path=RESULT_PATH):
    if not os.path.exists(result_path):
        print("No results yet:", result_path)
        return
    lines = open(result_path, "r", encoding="utf-8").read().strip().splitlines()
    for line in lines[-n:]:
        obj = json.loads(line)
        print("\n---")
        print("ok:", obj.get("ok"), "id:", obj.get("id"), "step:", obj.get("step"), "time:", obj.get("time"))
        if obj.get("ok"):
            print(obj.get("gen_text_stripped", obj.get("gen_text", ""))[:2000])
        else:
            print("error:", obj.get("error"))

read_last_results(3)
```

In [ ]:
# =======================
# 9) 配置并启动 GRPO
# =======================
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer

vllm_sampling_params = SamplingParams(
    temperature = temperature,
    top_p = top_p,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    learning_rate = 2e-6,
    warmup_ratio = 0.1,
    weight_decay = 0.001,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 10,

    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,

    num_generations = num_generations,
    max_prompt_length = 2048,
    max_completion_length = max_seq_length - max_prompt_length,

    max_steps = max_steps,
    save_steps = save_steps,

    beta = beta_kl,   # 小 KL 牵引（非 0 才会启用 KL 项）
    report_to = "none",
    output_dir = "grpo_outputs",
)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [reward_embedding_similarity],
    args = training_args,
    train_dataset = raw,
)

# 加上“按需生成”callback
trainer.add_callback(OnDemandGenerateCallback(
    tokenizer=tokenizer,
    request_path="grpo_eval_requests.jsonl",
    result_path="grpo_eval_results.jsonl",
    poll_interval_sec=5.0,       # 想更“随时”可设 2.0
    max_requests_per_poll=1,     # 单卡建议 1
    default_gen_kwargs={         # 默认生成参数（你也可以在请求里覆盖 gen）
        "max_new_tokens": 192,
        "temperature": 0.7,
        "top_p": 0.8,
        "top_k": 20,
        "do_sample": True,
    },
    enable_thinking=False, # Qwen3-4B-Thinking-2507的模型卡说过它会始终输出思维链，所以不需要设置True
    start_from_end=True,         # 重启训练后从 requests 文件末尾开始读（不回放旧请求）
    print_outputs=True,          # 训练日志里直接看到输出
    keep_full_text=True, # True: 保存 full_text（含prompt）; False: 仅保存生成部分
))

trainer.train()
# trainer.train(resume_from_checkpoint=True)


In [ ]:
# =======================
# 10) 保存 LoRA（你后续可合并导出整模）
# =======================
model.save_lora("grpo_lora_adapter")
print("Saved to grpo_lora_adapter/")


In [ ]:
# （可选）合并保存整模（按你的需求选择 merged_16bit / merged_4bit 等）
model.save_pretrained_merged("grpo_merged_model", tokenizer, save_method="merged_16bit")


## 11) 导出 GGUF（q4_k_m）
说明：
- 下面代码默认你此时的 `model` 仍然是 Unsloth 的模型对象，并且 GRPO 的 LoRA 还挂在其上（最推荐的导出时机）。
- 如果你已经重启 kernel、只剩下 `grpo_lora_adapter/`，看下一个单元格的“恢复后导出”版本。


In [ ]:
# =======================
# 11A) 直接导出 GGUF q4_k_m（推荐：训练结束立刻做）
# =======================

GGUF_DIR = "gguf_q4_k_m"   # 输出目录名
# quantization_method 可选： "q4_k_m" / "q8_0" / "f16" 等
model.save_pretrained_gguf(
    GGUF_DIR,
    tokenizer,
    quantization_method = "q4_k_m",
)

print("GGUF exported to:", GGUF_DIR)
# 生成的文件通常在该目录下，类似：*Q4_K_M*.gguf


In [ ]:
import os

# 防止从hf下载模型，直接用本地模型不联网，否则连不上hf导致失败
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"


# =======================
# 11B) 如果只剩 LoRA adapter：加载并导出 GGUF q4_k_m
# =======================

from unsloth import FastLanguageModel

LORA_PATH       = "grpo_lora_adapter"     # 你保存的 GRPO LoRA 目录
GGUF_DIR        = "gguf_q4_k_m"

# 加载lora模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LORA_PATH,
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = False,  # 导出 GGUF 不需要 vLLM rollout；关掉可少占资源
    # local_files_only = True,
)

# 导出 GGUF q4_k_m
model.save_pretrained_gguf(
    GGUF_DIR,
    tokenizer,
    quantization_method = "q4_k_m",
)

print("GGUF exported to:", GGUF_DIR)

In [ ]:
messages = [
    # {"role" : "user", "content" : "写个故事，讲述了一只小蝌蚪找妈妈的故事。"}
    {"role" : "user", "content" : "写个故事，讲述了一只小蝌蚪找妈妈的故事。\n\n写作规则：先给出一段 Markdown 引用块 `> **综述**`；若内容较长可分段输出，每次只写一段正文并在文末给出 `> **前文提要**`（概括这一段）。只有当全文结束时，最后另起一行写“（完）”。"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Qwen3-4B-Thinking-2507的模型卡说过它会始终输出思维链，所以不需要设置True
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 8192,
    # temperature = 1.5, min_p = 0.1,
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)